# Prova prova sa sa 

In [ ]:
!pip install pyhealth

In [3]:
import pandas as pd 

df = pd.read_csv('data/DIAGNOSES_ICD.csv')

In [4]:
from mimic3 import MIMIC3Dataset

dataset = MIMIC3Dataset(
    root='data/',
    tables=["DIAGNOSES_ICD"],
    code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
)


In [5]:
dataset.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n'

In [6]:
# get patient dictionary
patient_dict = dataset.patients
print(list(patient_dict.keys())[:10])

['10', '100', '1000', '10000', '10001', '10002', '10003', '10004', '10005', '10006']


In [7]:
# get the "10" patient
patient = patient_dict["10"]
patient.gender, patient.birth_datetime, patient.ethnicity, patient.death_datetime, patient.visits
print(patient.visits)

OrderedDict([('184167', Visit 184167 from patient 10 with 5 events from tables ['DIAGNOSES_ICD'])])


In [8]:
# get the visit list of this patient
visit_dict = patient.visits
print (list(visit_dict.keys()))

['184167']


In [9]:
# get the first visit
visit = visit_dict['184167']
visit.encounter_time, visit.available_tables, visit.num_events, visit.event_list_dict

(datetime.datetime(2103, 6, 28, 11, 36),
 ['DIAGNOSES_ICD'],
 5,
 {'DIAGNOSES_ICD': [Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
   Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
   Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
   Event with ICD9CM code V290 from table DIAGNOSES_ICD]})

In [10]:
visit.get_event_list('DIAGNOSES_ICD')

[Event with ICD9CM code V3000 from table DIAGNOSES_ICD,
 Event with ICD9CM code 7742 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76525 from table DIAGNOSES_ICD,
 Event with ICD9CM code 76515 from table DIAGNOSES_ICD,
 Event with ICD9CM code V290 from table DIAGNOSES_ICD]

In [50]:
import pandas as pd
import os

table = "NOTEEVENTS"

# read table
df = pd.read_csv(
    "data/NOTEEVENTS.csv",
    dtype={"SUBJECT_ID": str, "HADM_ID": str, "CATEGORY": str,"CHARTDATE": str, "TEXT": str},
    parse_dates=["CHARTDATE"],
    infer_datetime_format=True
)


C:\Users\AR\AppData\Local\Temp\ipykernel_13520\1121368045.py:7: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [51]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN\n\n\n#1 Infant remains in RA with O2 sats...
2083176,2070658,31097,115637,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,"Neonatology\nDOL #5, CGA 36 weeks.\n\nCVR: Con..."
2083177,2070659,31097,115637,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note\nFamily meeting held with ...
2083178,2070660,31097,115637,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN 1800\n\n\n#1 Resp: [**Known lastname 2243*...


In [52]:
import re
from nltk.corpus import stopwords

def tokenize_and_lemmatize(text):
    # Tokenizzazione delle parole
    tokens = word_tokenize(text)
    
    # Inizializzazione del lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Rimozione delle stopwords e simboli
    stop_words = set(stopwords.words('english'))
    symbols = ['[', '*', '+']
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words and not re.match(r'^\W+$', token) and not any(symbol in token for symbol in symbols)]
    
    # Lemmatizzazione delle parole
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Riaccorpamento del testo lemmatizzato
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

In [53]:
# filter for 'DISCHARGE SUMMARY' in the 'CATEGORY' column
df = df[df['CATEGORY'] == 'Discharge summary']
# drop records of the other patients
df = df[df["SUBJECT_ID"].isin(patient_dict.keys())]
# drop rows with missing values
df = df.dropna(subset=["SUBJECT_ID", "HADM_ID", "TEXT"])
# group by patient and visit
group_df = df.groupby("SUBJECT_ID")

In [54]:
df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...
...,...,...,...,...,...,...,...,...,...,...,...
60411,59608,93924,135672,2104-06-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 18321**],[**Known fir..."
60412,59609,17782,155710,2112-03-31,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname 2706**], [**Known fir..."
60413,59621,96333,183951,2197-06-18,NaN,NaN,Discharge summary,Addendum,NaN,NaN,Name: [**Known lastname 18357**]-[**Known las...
60414,59622,66717,169165,2129-08-14,NaN,NaN,Discharge summary,Addendum,NaN,NaN,"Name: [**Known lastname **],[**Known firstnam..."


In [57]:
# Define the ranges for subject IDs
ranges = [(10000, 20000), (20000, 30000), (30000, 40000)]

# Filter 5 patients from each range
selected_patients = []
for start, end in ranges:
    patients_in_range = df[(df['SUBJECT_ID'].astype(int) > start) & (df['SUBJECT_ID'].astype(int) <= end)]
    selected_patients.extend(patients_in_range.sample(5).to_dict('records'))

# Convert the list of dicts to a dataframe
selected_patients_df = pd.DataFrame(selected_patients)



In [58]:
selected_patients_list = selected_patients_df['SUBJECT_ID'].tolist()
selected_patients_list

['19860',
 '14026',
 '10487',
 '14261',
 '17982',
 '23065',
 '28479',
 '23349',
 '21356',
 '25928',
 '31627',
 '30114',
 '32332',
 '32058',
 '30782']

In [59]:
filtered_noteevents_df = df[df['SUBJECT_ID'].isin(selected_patients_list)]

In [60]:
filtered_noteevents_df

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
870,853,30114,192860,2116-06-10,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2116-6-1**] D...
871,854,30114,101731,2117-04-03,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2117-3-30**] ...
7868,10485,19860,183847,2152-09-07,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2152-8-29**] Dischar...
8773,8573,23065,153594,2175-07-21,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2175-7-15**] Discharge...
10958,6823,32058,108927,2193-03-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2193-3-28**] ...
11253,7557,32332,199767,2129-06-12,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2129-6-3**] D...
14964,14052,10487,185515,2134-04-08,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-3-29**] Dischar...
14965,14053,10487,188380,2134-10-16,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2134-10-12**] Discha...
14990,14078,10487,108477,2136-12-05,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2136-12-1**] ...
14991,14079,10487,199591,2136-12-27,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2136-12-22**] ...


In [97]:
#unique_subject_ids = noteevents_df['SUBJECT_ID'].unique()[:10]
#filtered_noteevents_df = noteevents_df[noteevents_df['SUBJECT_ID'].isin(unique_subject_ids)]

C:\Users\AR\AppData\Local\Temp\ipykernel_15480\2866892490.py:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  filtered_noteevents_df = df[df['SUBJECT_ID'] == (selected_patients_df)]


In [61]:
from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT

c:\Users\AR\AppData\Local\Programs\Python\Python310\lib\site-packages\medcat\cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [62]:
DATA_DIR = "./data_med/"
vocab_path = DATA_DIR + "vocab.dat"
cdb_path = DATA_DIR + "cdb-medmen.dat"

In [63]:
# Create and load the CDB (Concept Database)
cdb = CDB.load(cdb_path)

# Create and load the Vocabulary
vocab = Vocab.load(vocab_path)

# Load model for MetaAnnotations (optional)
mc_status = MetaCAT.load("./data_med/Status/")

In [64]:
# Create CAT - the main class from medcat used for concept annotation
cat = CAT(cdb=cdb, config=cdb.config, vocab=vocab, meta_cats=[mc_status])

In [65]:
# Detect only the following entity:
    # bpoc|T023|Body Part, Organ, or Organ Component
    # lbtr|T034|Laboratory or Test Result
    # patf|T046|Pathologic Function
    # dsyn|T047|Disease or Syndrome
    # mobd|T048|Mental or Behavioral Dysfunction
    # lbpr|T059|Laboratory Procedure
    # diap|T060|Diagnostic Procedure
    # topp|T061|Therapeutic or Preventive Procedure
    # phsu|T121|Pharmacologic Substance ---> SE LO AGGIUNGO MI METTE ANCHE ALTRA ROBA TIPO VITAMINE, ENZIMI, ETC.
    # sosy|T184|Sign or Symptom

type_ids_filter = ['T047','T184']
cui_filters = set()
for type_ids in type_ids_filter:
    cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters

In [66]:
color_dict = {
    'Body Part, Organ, or Organ Component': '#FFCE80',  # Arancione pastello
    'Laboratory or Test Result': '#FFF9C4',             # Giallo chiaro
    'Disease or Syndrome': '#B5EAD7',                   # Verde acqua chiaro
    'Mental or Behavioral Dysfunction': '#F0B2FF',      # Viola pastello
    'Laboratory Procedure': '#D0D9FF',                  # Blu chiaro
    'Diagnostic Procedure': '#FFD9EC',                  # Rosa chiaro
    'Therapeutic or Preventive Procedure': '#C4FFFF',   # Azzurro chiaro
    'Pharmacologic Substance': '#FFDAB9',               # Pesca pastello
    'Sign or Symptom': '#FFC4F3',                       # Magenta chiaro
    'Pathologic Function': '#B5EAD7'                    # Verde acqua chiaro
}

In [77]:
entities_lemma = cat.get_entities(filtered_noteevents_df['TEXT'][870])

In [78]:
extracted_data = []
# Extract 'pretty_name' and 'types' values from the dictionary
for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        if value['types'][0] == 'Pharmacologic Substance':
            extracted_data.append((value['source_value'], value['types'][0]))
        else:
            extracted_data.append((value['pretty_name'], value['types'][0]))

# Create a dataframe
df_extra = pd.DataFrame(extracted_data, columns=['Nome', 'Entità'])


In [79]:
from spacy import displacy

doc = {"text": df['TEXT'][0], "ents": [], "title": None}

for key, value in entities_lemma['entities'].items():
    if value['meta_anns']['Status']['value'] == 'Affirmed':
        ent = {
            'start': value['start'],
            'end': value['end'],
            'label': value['types'][0]
        }
        doc['ents'].append(ent)

colors = color_dict
options = {"ents": list(colors.keys()), "colors": colors}

# Call spacy.displacy.render() to visualize the entities
displacy.render(doc, style='ent', options=options, jupyter=True, manual=True)